In [71]:
import cmipld
# from cmipld.utils.offline import LD_server, replace_loader
from pyld import jsonld

In [19]:
import cmipld
import importlib


repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

whoami = cmipld.reverse_mapping()[io_url]
print('-'*50)
print(f'Parsing repo: {whoami}')
print(f'Location: {repo_url}')
print(f'Github IO link: {io_url}')
print('-'*50)

'''
To keep this set of functions generic, we create separate files containing all relevant functions for a repository to be processed. 
When running this script we determine the parent repository, and consequently its prefix. This is then used to grab all relevant functions for processing. 

Example usage: getattr(this,'activity')({'activity':[{"label":1,"id":'one'}]})

'''
# Dynamically import the target module and assign it to a variable
module_path = f"cmipld.cvs.{whoami}" # go to CMIPLD/CVs/<value of whoami>.py for functions
this = importlib.import_module(module_path)



make test for url2io
--------------------------------------------------
Parsing repo: cmip6plus
Location: https://github.com/WCRP-CMIP/CMIP6Plus_CVs
Github IO link: https://wcrp-cmip.github.io/CMIP6Plus_CVs/
--------------------------------------------------


In [22]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 
cmipld.processor.clear_cache()

es_data = {}

for value in ['experiment', 'source']:
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}experiment/graph.jsonld') 
    result = es_data[value].frame({'@type':[f'wcrp:{value}'],'@id':f'{io_url}experiment/abrupt-4xco2'})
    
    a = es_data[value]
    break

result

Context fetch failed: Expecting property name enclosed in double quotes: line 16 column 17 (char 529)


Error extracting dependencies: ('Could not expand input before framing.',)
Type: jsonld.FrameError
Cause: ('Dereferencing a URL did not result in a valid JSON-LD object. Possible causes are an inaccessible URL perhaps due to a same-origin policy (ensure the server uses CORS if you are using client-side JavaScript), too many redirects, a non-JSON response, or more than one HTTP Link Header was provided for a remote context.',)
Type: jsonld.InvalidUrl
Code: loading remote context failed
Details: {'url': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/_context_', 'cause': JsonLdError('Could not retrieve a JSON-LD document from the URL.')}  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/pyld/jsonld.py", line 1032, in frame
    expanded = self.expand(input_, options)
  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/pyld/jsonld.py", line 870, in expand
    expanded = self._expand(act

ContextResolutionError: Fetch error: Expecting property name enclosed in double quotes: line 16 column 17 (char 529)

In [73]:
# # Set up offline override for the current repository

# ldpath = cmipld.utils.git.ldpath()


# # create a temporary directory containing (a copy of) the current repository
# # localserver = LD_server(copy = [[ldpath,'current_repo']])
# repos= { 
#         'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
#         'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
#         'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
#         }

# localserver = LD_server(repos = repos.items(),copy = [[ldpath,whoami]],override='y')

# # localserver = LD_server(copy = [[ldpath,'cmip6plus']])


# localhost = localserver.start_server(8080)


In [74]:
# # 
# replace_loader(localhost,[list(i) for i in repos.items()])
# # 

In [75]:
from collections import OrderedDict

CV = OrderedDict()




In [ ]:

for item in [
    'repo',
    'mip-era',
    'license',
    'activity',
    'drs',
    'product',
    'required-global-attributes',
    # 'tables',
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    ld  = cmipld.processor.expand_document(f'{io_url}project/{item}.json', depth=3)[0]
    # functions may be autogenrated in (reponame).py under the CVs dir. 
    CV[item] = getattr(this,item)(ld)
    


In [77]:
# adding the index properties
CV.update(cmipld.processor.get(f'{io_url}/project/index.json')[0]['index'])

In [ ]:
CV

In [ ]:
print('make a file for relevant realms and relevant realms')

for universe_item in ['realm','frequency']:
    # 'grid-label''nominal'
    print(universe_item)
    path = f'{cmipld.mapping["universal"]}{universe_item}/graph.jsonld'
    ld  = cmipld.jsonld.compact(path,path)['@graph']
    CV[universe_item] = getattr(this,universe_item)(ld)

In [ ]:
# path = f'{cmipld.mapping["universal"]}resolution/graph.jsonld'
# data  = cmipld.jsonld.compact(path,path)['@graph']

# CV['nominal_resolution'] = [f"{r['value']} {r['unit']}" for r in sorted(data, key=lambda x: float(x['value']) if x['value'].replace('.', '', 1).isdigit() else float('inf'))]

print('make a file for relevant nominal resolution')

In [ ]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 

es_data = {}

for value in ['experiment', 'source']:
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}experiment/graph.jsonld') 
    result = es_data[value].frame({'@type':[f'wcrp:{value}']})
    
    a = es_data[value]
    break

result[1]

In [ ]:
from pyld import jsonld
# jsonld.frame(es_data[value].corpus,{'@type':[f'wcrp:{value}'],'@context':es_data[value].context,'@embed':'@always'})['@graph'][3]['model_realms']

ctx = {'@base': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/',
 '@vocab': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/',
 'id': '@id',
 'type': '@type',
 'cf': 'https://wcrp-cmip.github.io/CF/',
 'cmip6plus': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/',
 'cmip7': 'https://wcrp-cmip.github.io/CMIP7_CVs/',
 'obs4mips': 'https://wolfiex.github.io/obs4MIPs-cmor-tables-ld/',
 'universal': 'https://wcrp-cmip.github.io/WCRP-universe/',
 'variables': 'https://wcrp-cmip.github.io/MIP-variables/',
 'activity': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'},
#  "model_realms": {
#                 "@container": [
#                     "@id"
#                 ],
#                 # '@type':'@id',
#                 # "@id": "https://test/{@index}",
#                 "@context": "https://wcrp-cmip.github.io/WCRP-universe/source-type/_context_"
#             },
 'parent_activity': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'},
 'parent_experiment': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment/_context_'},
 'sub_experiment': {'@type': '@id',
  '@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/sub-experiment/_context_'},
#  "entries": "@none"
}
#

jsonld.frame(a.url,{'@type':[f'wcrp:experiment'],'@context':ctx})['@graph'][1]

In [ ]:
a.context

In [ ]:
jsonld.expand(f'{io_url}experiment/abrupt-4xco2')

In [ ]:
[i for i in jsonld.frame(f'{io_url}experiment/graph.jsonld',{"@explicit":True})['@graph'] if '@type' in i]

In [ ]:
cmipld.processor.clear_cache()

jsonld.frame(es_data[value].corpus,{'@context':es_data[value].context,'@embed':'@always'})['@graph'][1]

In [ ]:
import json
"@container" in json.dumps(cmipld.processor.loader( es_data[value].context,{} )['document']['@context'])

In [ ]:
cmipld.processor.loader(es_data[value].url,{})['document']['@graph'][0]

In [ ]:
result[0]
es_data[value].container  #.corpus['@graph'][1]

g = es_data[value].corpus['@graph']

In [ ]:
# corpus = {}
# for i in g:
    
result[3]

In [86]:
[]


item = result[1]
item_id = item['experiment_id'] = item['label']

item['activity_id'] = this.get_entry(item['activity'])

# check that activity in activity list

item['additional_allowed_model_components'] = []
item['required_model_components'] = []

# for item in item['model_realms']



In [ ]:
this.name_entry(list(item['model_realms'].values()),'is_required')

In [ ]:
item['parent_experiment']
item_id

In [ ]:
help(jsonld.frame)

In [ ]:
CV

In [91]:
# e1 = exp[1]
# # del e1['id'], e1['type']
# for k,v in e1.items():
#     if isinstance(v,dict) and 'type' in v:
#         print(k)
#         e1[k] = v['label']
# e1
